# Comprehensive Machine Learning Analysis for Student Performance Prediction

This notebook demonstrates a complete machine learning workflow including:
- Data preprocessing and feature engineering
- Multiple ML algorithms comparison
- Comprehensive model evaluation
- Advanced visualizations with justifications
- Performance optimization techniques

## Table of Contents
1. [Data Loading and Exploration](#data-loading)
2. [Data Preprocessing](#preprocessing)
3. [Feature Engineering](#feature-engineering)
4. [Model Training and Comparison](#model-training)
5. [Model Evaluation](#evaluation)
6. [Advanced Analysis](#advanced-analysis)
7. [Conclusions and Recommendations](#conclusions)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

## 1. Data Loading and Exploration <a id="data-loading"></a>

We'll generate a comprehensive synthetic dataset that simulates real-world student performance data with various cognitive skills and environmental factors.

In [ ]:
def generate_comprehensive_dataset(n_samples=5000):
    """
    Generate a comprehensive synthetic dataset for student performance analysis.
    
    Parameters:
    n_samples (int): Number of samples to generate
    
    Returns:
    pd.DataFrame: Generated dataset
    """
    np.random.seed(42)
    
    # Generate base features
    data = {
        'student_id': range(1, n_samples + 1),
        'age': np.random.normal(16, 2, n_samples).clip(13, 20),
        'grade_level': np.random.choice([9, 10, 11, 12], n_samples, p=[0.25, 0.25, 0.25, 0.25]),
        'gender': np.random.choice(['Male', 'Female', 'Other'], n_samples, p=[0.45, 0.45, 0.1]),
        'socioeconomic_status': np.random.choice(['Low', 'Medium', 'High'], n_samples, p=[0.3, 0.5, 0.2]),
        
        # Cognitive skills (0-100 scale)
        'comprehension': np.random.beta(2, 2, n_samples) * 100,
        'attention_span': np.random.beta(2, 2, n_samples) * 100,
        'memory_retention': np.random.beta(2, 2, n_samples) * 100,
        'problem_solving': np.random.beta(2, 2, n_samples) * 100,
        'critical_thinking': np.random.beta(2, 2, n_samples) * 100,
        
        # Study habits and environment
        'study_hours_per_week': np.random.gamma(2, 5, n_samples).clip(0, 50),
        'sleep_hours_per_night': np.random.normal(7.5, 1.5, n_samples).clip(4, 12),
        'extracurricular_activities': np.random.poisson(2, n_samples).clip(0, 8),
        'family_support': np.random.beta(3, 2, n_samples) * 10,
        'teacher_rating': np.random.beta(3, 2, n_samples) * 10,
        
        # Technology and resources
        'computer_access': np.random.choice([0, 1], n_samples, p=[0.2, 0.8]),
        'internet_quality': np.random.choice(['Poor', 'Fair', 'Good', 'Excellent'], n_samples, p=[0.1, 0.2, 0.4, 0.3]),
        'learning_resources': np.random.beta(2, 2, n_samples) * 10,
    }
    
    df = pd.DataFrame(data)
    
    # Create realistic correlations for assessment scores
    # Assessment score is influenced by multiple factors with realistic noise
    assessment_base = (
        0.3 * df['comprehension'] +
        0.25 * df['problem_solving'] +
        0.2 * df['critical_thinking'] +
        0.15 * df['attention_span'] +
        0.1 * df['memory_retention'] +
        0.05 * df['study_hours_per_week'] +
        0.03 * df['family_support'] * 10 +
        0.02 * df['teacher_rating'] * 10
    )
    
    # Add socioeconomic and demographic effects
    ses_effect = df['socioeconomic_status'].map({'Low': -5, 'Medium': 0, 'High': 5})
    grade_effect = (df['grade_level'] - 9) * 2  # Higher grades slightly better
    
    # Add realistic noise and constraints
    noise = np.random.normal(0, 8, n_samples)
    df['assessment_score'] = (assessment_base + ses_effect + grade_effect + noise).clip(0, 100)
    
    # Create engagement metrics
    df['class_participation'] = (df['attention_span'] * 0.6 + df['family_support'] * 4 + np.random.normal(0, 10, n_samples)).clip(0, 100)
    df['homework_completion'] = (df['study_hours_per_week'] * 1.5 + df['family_support'] * 3 + np.random.normal(0, 8, n_samples)).clip(0, 100)
    
    return df

# Generate the dataset
df = generate_comprehensive_dataset(5000)
print(f"Dataset generated with {len(df)} samples and {len(df.columns)} features")
print("\nDataset shape:", df.shape)
print("\nFirst few rows:")
df.head()

In [ ]:
# Dataset overview and basic statistics
print("=== DATASET OVERVIEW ===")
print(f"Total samples: {len(df)}")
print(f"Total features: {len(df.columns)}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n=== DATA TYPES ===")
print(df.dtypes)

print("\n=== MISSING VALUES ===")
missing_values = df.isnull().sum()
if missing_values.sum() == 0:
    print("No missing values found!")
else:
    print(missing_values[missing_values > 0])

print("\n=== BASIC STATISTICS ===")
df.describe()

### Data Visualization - Exploratory Data Analysis

**Graph Justification**: The following visualizations help us understand:
1. **Distribution plots**: Identify data skewness and outliers
2. **Correlation heatmap**: Understand feature relationships
3. **Box plots**: Compare distributions across categories
4. **Scatter plots**: Visualize relationships between key variables

In [ ]:
# Create comprehensive EDA visualizations
fig, axes = plt.subplots(3, 3, figsize=(20, 15))
fig.suptitle('Exploratory Data Analysis - Key Features Distribution', fontsize=16, fontweight='bold')

# 1. Assessment Score Distribution
axes[0, 0].hist(df['assessment_score'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Assessment Score Distribution')
axes[0, 0].set_xlabel('Assessment Score')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['assessment_score'].mean(), color='red', linestyle='--', label=f'Mean: {df["assessment_score"].mean():.1f}')
axes[0, 0].legend()

# 2. Cognitive Skills Distribution
cognitive_skills = ['comprehension', 'attention_span', 'memory_retention', 'problem_solving', 'critical_thinking']
df[cognitive_skills].boxplot(ax=axes[0, 1])
axes[0, 1].set_title('Cognitive Skills Distribution')
axes[0, 1].set_ylabel('Score (0-100)')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Study Hours vs Assessment Score
axes[0, 2].scatter(df['study_hours_per_week'], df['assessment_score'], alpha=0.6, color='green')
axes[0, 2].set_title('Study Hours vs Assessment Score')
axes[0, 2].set_xlabel('Study Hours per Week')
axes[0, 2].set_ylabel('Assessment Score')

# 4. Socioeconomic Status Impact
sns.boxplot(data=df, x='socioeconomic_status', y='assessment_score', ax=axes[1, 0])
axes[1, 0].set_title('Assessment Score by Socioeconomic Status')
axes[1, 0].set_ylabel('Assessment Score')

# 5. Grade Level Performance
grade_performance = df.groupby('grade_level')['assessment_score'].mean()
axes[1, 1].bar(grade_performance.index, grade_performance.values, color='orange', alpha=0.7)
axes[1, 1].set_title('Average Assessment Score by Grade Level')
axes[1, 1].set_xlabel('Grade Level')
axes[1, 1].set_ylabel('Average Assessment Score')

# 6. Sleep vs Performance
axes[1, 2].scatter(df['sleep_hours_per_night'], df['assessment_score'], alpha=0.6, color='purple')
axes[1, 2].set_title('Sleep Hours vs Assessment Score')
axes[1, 2].set_xlabel('Sleep Hours per Night')
axes[1, 2].set_ylabel('Assessment Score')

# 7. Family Support Impact
axes[2, 0].scatter(df['family_support'], df['assessment_score'], alpha=0.6, color='red')
axes[2, 0].set_title('Family Support vs Assessment Score')
axes[2, 0].set_xlabel('Family Support (0-10)')
axes[2, 0].set_ylabel('Assessment Score')

# 8. Technology Access Impact
tech_performance = df.groupby('computer_access')['assessment_score'].mean()
axes[2, 1].bar(['No Computer', 'Has Computer'], tech_performance.values, color=['red', 'green'], alpha=0.7)
axes[2, 1].set_title('Assessment Score by Computer Access')
axes[2, 1].set_ylabel('Average Assessment Score')

# 9. Age Distribution
axes[2, 2].hist(df['age'], bins=20, alpha=0.7, color='brown', edgecolor='black')
axes[2, 2].set_title('Age Distribution')
axes[2, 2].set_xlabel('Age')
axes[2, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("\n=== KEY INSIGHTS FROM EDA ===")
print(f"1. Average assessment score: {df['assessment_score'].mean():.2f} ± {df['assessment_score'].std():.2f}")
print(f"2. Study hours correlation with performance: {df['study_hours_per_week'].corr(df['assessment_score']):.3f}")
print(f"3. Sleep hours correlation with performance: {df['sleep_hours_per_night'].corr(df['assessment_score']):.3f}")
print(f"4. Family support correlation with performance: {df['family_support'].corr(df['assessment_score']):.3f}")

In [ ]:
# Correlation Analysis
# **Graph Justification**: Correlation heatmap reveals multicollinearity and helps with feature selection

# Select numerical columns for correlation analysis
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols.remove('student_id')  # Remove ID column

# Calculate correlation matrix
correlation_matrix = df[numerical_cols].corr()

# Create correlation heatmap
plt.figure(figsize=(16, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.2f')
plt.title('Feature Correlation Matrix\n(Upper triangle masked for clarity)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Identify highly correlated features with target
target_correlations = correlation_matrix['assessment_score'].abs().sort_values(ascending=False)
print("\n=== FEATURES MOST CORRELATED WITH ASSESSMENT SCORE ===")
for feature, corr in target_correlations.head(10).items():
    if feature != 'assessment_score':
        print(f"{feature}: {corr:.3f}")

## 2. Data Preprocessing <a id="preprocessing"></a>

Comprehensive preprocessing pipeline including:
- Handling categorical variables
- Feature scaling
- Outlier detection and treatment
- Data validation

In [ ]:
def comprehensive_preprocessing(df):
    """
    Comprehensive preprocessing pipeline for the student dataset.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    
    Returns:
    tuple: (processed_df, encoders, scalers)
    """
    df_processed = df.copy()
    encoders = {}
    scalers = {}
    
    print("=== PREPROCESSING PIPELINE ===")
    
    # 1. Handle categorical variables
    print("1. Encoding categorical variables...")
    
    # Label encoding for ordinal variables
    ordinal_mappings = {
        'socioeconomic_status': {'Low': 0, 'Medium': 1, 'High': 2},
        'internet_quality': {'Poor': 0, 'Fair': 1, 'Good': 2, 'Excellent': 3}
    }
    
    for col, mapping in ordinal_mappings.items():
        df_processed[col] = df_processed[col].map(mapping)
        encoders[col] = mapping
    
    # One-hot encoding for nominal variables
    nominal_cols = ['gender']
    for col in nominal_cols:
        dummies = pd.get_dummies(df_processed[col], prefix=col, drop_first=True)
        df_processed = pd.concat([df_processed, dummies], axis=1)
        df_processed.drop(col, axis=1, inplace=True)
        encoders[col] = list(dummies.columns)
    
    print(f"   - Encoded {len(ordinal_mappings) + len(nominal_cols)} categorical variables")
    
    # 2. Outlier detection and treatment
    print("2. Detecting and treating outliers...")
    
    numerical_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
    numerical_cols.remove('student_id')
    
    outlier_counts = {}
    for col in numerical_cols:
        Q1 = df_processed[col].quantile(0.25)
        Q3 = df_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = (df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)
        outlier_counts[col] = outliers.sum()
        
        # Cap outliers instead of removing them
        df_processed[col] = df_processed[col].clip(lower_bound, upper_bound)
    
    total_outliers = sum(outlier_counts.values())
    print(f"   - Detected and capped {total_outliers} outliers across all features")
    
    # 3. Feature scaling
    print("3. Scaling numerical features...")
    
    # Separate features and target
    feature_cols = [col for col in df_processed.columns if col not in ['student_id', 'assessment_score']]
    
    scaler = StandardScaler()
    df_processed[feature_cols] = scaler.fit_transform(df_processed[feature_cols])
    scalers['features'] = scaler
    
    print(f"   - Scaled {len(feature_cols)} features using StandardScaler")
    
    # 4. Data validation
    print("4. Validating processed data...")
    
    # Check for any remaining issues
    missing_after = df_processed.isnull().sum().sum()
    infinite_values = np.isinf(df_processed.select_dtypes(include=[np.number])).sum().sum()
    
    print(f"   - Missing values after preprocessing: {missing_after}")
    print(f"   - Infinite values after preprocessing: {infinite_values}")
    print(f"   - Final dataset shape: {df_processed.shape}")
    
    return df_processed, encoders, scalers

# Apply preprocessing
df_processed, encoders, scalers = comprehensive_preprocessing(df)

print("\n=== PREPROCESSING COMPLETE ===")
print(f"Original features: {len(df.columns)}")
print(f"Processed features: {len(df_processed.columns)}")
print(f"Feature columns: {len([col for col in df_processed.columns if col not in ['student_id', 'assessment_score']])}")

## 3. Feature Engineering <a id="feature-engineering"></a>

Creating new features to improve model performance:
- Interaction features
- Polynomial features
- Domain-specific features

In [ ]:
def create_engineered_features(df_processed):
    """
    Create engineered features based on domain knowledge.
    
    Parameters:
    df_processed (pd.DataFrame): Preprocessed dataframe
    
    Returns:
    pd.DataFrame: Dataframe with engineered features
    """
    df_engineered = df_processed.copy()
    
    print("=== FEATURE ENGINEERING ===")
    
    # 1. Cognitive Skills Composite Score
    cognitive_cols = ['comprehension', 'attention_span', 'memory_retention', 'problem_solving', 'critical_thinking']
    df_engineered['cognitive_composite'] = df_engineered[cognitive_cols].mean(axis=1)
    print("1. Created cognitive composite score")
    
    # 2. Study Efficiency (performance per study hour)
    df_engineered['study_efficiency'] = df_engineered['assessment_score'] / (df_engineered['study_hours_per_week'] + 1)
    print("2. Created study efficiency metric")
    
    # 3. Work-Life Balance Score
    df_engineered['work_life_balance'] = (df_engineered['sleep_hours_per_night'] * 0.4 + 
                                         (10 - df_engineered['extracurricular_activities']) * 0.3 + 
                                         (50 - df_engineered['study_hours_per_week']) * 0.3)
    print("3. Created work-life balance score")
    
    # 4. Support System Strength
    df_engineered['support_system'] = (df_engineered['family_support'] * 0.6 + 
                                      df_engineered['teacher_rating'] * 0.4)
    print("4. Created support system strength metric")
    
    # 5. Technology Advantage Score
    df_engineered['tech_advantage'] = (df_engineered['computer_access'] * 0.5 + 
                                      df_engineered['internet_quality'] * 0.3 + 
                                      df_engineered['learning_resources'] * 0.2)
    print("5. Created technology advantage score")
    
    # 6. Interaction Features
    df_engineered['comprehension_x_attention'] = df_engineered['comprehension'] * df_engineered['attention_span']
    df_engineered['study_hours_x_family_support'] = df_engineered['study_hours_per_week'] * df_engineered['family_support']
    print("6. Created interaction features")
    
    # 7. Age-Grade Alignment
    expected_age = df_engineered['grade_level'] + 5  # Assuming grade 9 = age 14
    df_engineered['age_grade_alignment'] = 1 - abs(df_engineered['age'] - expected_age) / 4
    print("7. Created age-grade alignment feature")
    
    print(f"\nFeature engineering complete. Added {len(df_engineered.columns) - len(df_processed.columns)} new features.")
    print(f"Total features now: {len(df_engineered.columns)}")
    
    return df_engineered

# Apply feature engineering
df_engineered = create_engineered_features(df_processed)

# Display new features
new_features = [col for col in df_engineered.columns if col not in df_processed.columns]
print(f"\nNew engineered features: {new_features}")

In [ ]:
# Feature Selection using statistical methods
# **Graph Justification**: Feature importance plots help identify the most predictive features

# Prepare data for feature selection
feature_cols = [col for col in df_engineered.columns if col not in ['student_id', 'assessment_score']]
X = df_engineered[feature_cols]
y = df_engineered['assessment_score']

# Statistical feature selection
selector = SelectKBest(score_func=f_regression, k=15)
X_selected = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()].tolist()

# Get feature scores
feature_scores = pd.DataFrame({
    'feature': X.columns,
    'score': selector.scores_,
    'selected': selector.get_support()
}).sort_values('score', ascending=False)

# Visualize feature importance
plt.figure(figsize=(14, 8))
colors = ['green' if selected else 'lightgray' for selected in feature_scores['selected']]
bars = plt.bar(range(len(feature_scores)), feature_scores['score'], color=colors, alpha=0.7)
plt.title('Feature Importance Scores (F-regression)\nGreen bars indicate selected features', fontsize=14, fontweight='bold')
plt.xlabel('Features')
plt.ylabel('F-Score')
plt.xticks(range(len(feature_scores)), feature_scores['feature'], rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("=== TOP 15 SELECTED FEATURES ===")
for i, (_, row) in enumerate(feature_scores.head(15).iterrows(), 1):
    status = "✓" if row['selected'] else "✗"
    print(f"{i:2d}. {status} {row['feature']:<25} (Score: {row['score']:.2f})")

print(f"\nSelected {len(selected_features)} features out of {len(feature_cols)} total features.")

## 4. Model Training and Comparison <a id="model-training"></a>

Training multiple machine learning algorithms and comparing their performance:
- Linear models (Linear Regression, Ridge, Lasso)
- Tree-based models (Random Forest, Gradient Boosting)
- Support Vector Regression
- Neural Networks

In [ ]:
# Prepare data for modeling
X = df_engineered[selected_features]
y = df_engineered['assessment_score']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=pd.cut(y, bins=5))

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")
print(f"Features used: {len(selected_features)}")

# Define models to compare
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(kernel='rbf', C=1.0),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

print(f"\nTraining {len(models)} different models...")

In [ ]:
# Train and evaluate all models
results = {}
trained_models = {}

print("=== MODEL TRAINING AND EVALUATION ===")

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train model
    model.fit(X_train, y_train)
    trained_models[name] = model
    
    # Make predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    
    results[name] = {
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'y_pred_test': y_pred_test
    }
    
    print(f"  Train R²: {train_r2:.4f} | Test R²: {test_r2:.4f}")
    print(f"  Train RMSE: {train_rmse:.4f} | Test RMSE: {test_rmse:.4f}")
    print(f"  CV R² (5-fold): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

print("\n=== MODEL COMPARISON COMPLETE ===")

In [ ]:
# Create comprehensive model comparison visualizations
# **Graph Justification**: These plots help compare model performance across multiple metrics

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Comprehensive Model Performance Comparison', fontsize=16, fontweight='bold')

# 1. R² Score Comparison
model_names = list(results.keys())
train_r2_scores = [results[name]['train_r2'] for name in model_names]
test_r2_scores = [results[name]['test_r2'] for name in model_names]

x = np.arange(len(model_names))
width = 0.35

axes[0, 0].bar(x - width/2, train_r2_scores, width, label='Train R²', alpha=0.8, color='skyblue')
axes[0, 0].bar(x + width/2, test_r2_scores, width, label='Test R²', alpha=0.8, color='lightcoral')
axes[0, 0].set_title('R² Score Comparison')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(model_names, rotation=45, ha='right')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. RMSE Comparison
train_rmse_scores = [results[name]['train_rmse'] for name in model_names]
test_rmse_scores = [results[name]['test_rmse'] for name in model_names]

axes[0, 1].bar(x - width/2, train_rmse_scores, width, label='Train RMSE', alpha=0.8, color='lightgreen')
axes[0, 1].bar(x + width/2, test_rmse_scores, width, label='Test RMSE', alpha=0.8, color='orange')
axes[0, 1].set_title('RMSE Comparison (Lower is Better)')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(model_names, rotation=45, ha='right')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Cross-Validation Scores
cv_means = [results[name]['cv_mean'] for name in model_names]
cv_stds = [results[name]['cv_std'] for name in model_names]

axes[0, 2].bar(model_names, cv_means, yerr=cv_stds, capsize=5, alpha=0.8, color='purple')
axes[0, 2].set_title('Cross-Validation R² Scores (5-fold)')
axes[0, 2].set_ylabel('CV R² Score')
axes[0, 2].tick_params(axis='x', rotation=45)
axes[0, 2].grid(True, alpha=0.3)

# 4. Actual vs Predicted for best model
best_model_name = max(results.keys(), key=lambda x: results[x]['test_r2'])
best_predictions = results[best_model_name]['y_pred_test']

axes[1, 0].scatter(y_test, best_predictions, alpha=0.6, color='blue')
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_title(f'Actual vs Predicted - {best_model_name}')
axes[1, 0].set_xlabel('Actual Assessment Score')
axes[1, 0].set_ylabel('Predicted Assessment Score')
axes[1, 0].grid(True, alpha=0.3)

# 5. Residuals plot for best model
residuals = y_test - best_predictions
axes[1, 1].scatter(best_predictions, residuals, alpha=0.6, color='green')
axes[1, 1].axhline(y=0, color='r', linestyle='--')
axes[1, 1].set_title(f'Residuals Plot - {best_model_name}')
axes[1, 1].set_xlabel('Predicted Assessment Score')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].grid(True, alpha=0.3)

# 6. Model Performance Summary
performance_df = pd.DataFrame(results).T[['test_r2', 'test_rmse', 'cv_mean']]
performance_df.plot(kind='bar', ax=axes[1, 2], alpha=0.8)
axes[1, 2].set_title('Model Performance Summary')
axes[1, 2].set_ylabel('Score')
axes[1, 2].tick_params(axis='x', rotation=45)
axes[1, 2].legend(['Test R²', 'Test RMSE', 'CV Mean R²'])
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed results table
print("\n=== DETAILED MODEL PERFORMANCE RESULTS ===")
results_df = pd.DataFrame(results).T
results_df = results_df[['test_r2', 'test_rmse', 'test_mae', 'cv_mean', 'cv_std']]
results_df.columns = ['Test R²', 'Test RMSE', 'Test MAE', 'CV Mean R²', 'CV Std R²']
results_df = results_df.round(4)
results_df = results_df.sort_values('Test R²', ascending=False)

print(results_df)

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Test R²: {results[best_model_name]['test_r2']:.4f}")
print(f"   Test RMSE: {results[best_model_name]['test_rmse']:.4f}")
print(f"   CV R²: {results[best_model_name]['cv_mean']:.4f} ± {results[best_model_name]['cv_std']:.4f}")

## 5. Model Evaluation <a id="evaluation"></a>

Deep dive into the best performing model with:
- Feature importance analysis
- Learning curves
- Hyperparameter optimization
- Error analysis

In [ ]:
# Hyperparameter optimization for the best model
print(f"=== HYPERPARAMETER OPTIMIZATION FOR {best_model_name} ===")

if best_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    base_model = RandomForestRegressor(random_state=42)
    
elif best_model_name == 'Gradient Boosting':
    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 0.9, 1.0]
    }
    base_model = GradientBoostingRegressor(random_state=42)
    
else:
    # Default to Random Forest if other models are best
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10]
    }
    base_model = RandomForestRegressor(random_state=42)

# Perform grid search
print("Performing grid search with 3-fold cross-validation...")
grid_search = GridSearchCV(base_model, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Get best model
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_cv_score = grid_search.best_score_

print(f"\nBest parameters: {best_params}")
print(f"Best CV score: {best_cv_score:.4f}")

# Evaluate optimized model
y_pred_optimized = best_model.predict(X_test)
optimized_r2 = r2_score(y_test, y_pred_optimized)
optimized_rmse = np.sqrt(mean_squared_error(y_test, y_pred_optimized))

print(f"\nOptimized model performance:")
print(f"Test R²: {optimized_r2:.4f}")
print(f"Test RMSE: {optimized_rmse:.4f}")
print(f"Improvement in R²: {optimized_r2 - results[best_model_name]['test_r2']:.4f}")

In [ ]:
# Feature importance analysis for tree-based models
# **Graph Justification**: Feature importance helps understand which factors most influence student performance

if hasattr(best_model, 'feature_importances_'):
    # Get feature importances
    feature_importance = pd.DataFrame({
        'feature': selected_features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance.head(15), x='importance', y='feature', palette='viridis')
    plt.title(f'Top 15 Feature Importances - Optimized {best_model_name}', fontsize=14, fontweight='bold')
    plt.xlabel('Feature Importance')
    plt.ylabel('Features')
    plt.tight_layout()
    plt.show()
    
    print("=== TOP 10 MOST IMPORTANT FEATURES ===")
    for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
        print(f"{i:2d}. {row['feature']:<25} ({row['importance']:.4f})")
        
else:
    print("Feature importance not available for this model type.")

In [ ]:
# Learning curves analysis
# **Graph Justification**: Learning curves show if the model would benefit from more data or is overfitting

from sklearn.model_selection import learning_curve

print("Generating learning curves...")

# Generate learning curves
train_sizes, train_scores, val_scores = learning_curve(
    best_model, X_train, y_train, cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='r2'
)

# Calculate mean and std
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(12, 8))
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training Score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.plot(train_sizes, val_mean, 'o-', color='red', label='Validation Score')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')

plt.title(f'Learning Curves - Optimized {best_model_name}', fontsize=14, fontweight='bold')
plt.xlabel('Training Set Size')
plt.ylabel('R² Score')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Analysis of learning curves
final_train_score = train_mean[-1]
final_val_score = val_mean[-1]
gap = final_train_score - final_val_score

print(f"\n=== LEARNING CURVE ANALYSIS ===")
print(f"Final training score: {final_train_score:.4f}")
print(f"Final validation score: {final_val_score:.4f}")
print(f"Training-validation gap: {gap:.4f}")

if gap > 0.1:
    print("⚠️  Model shows signs of overfitting. Consider regularization or more data.")
elif gap < 0.05:
    print("✅ Model shows good generalization with minimal overfitting.")
else:
    print("✅ Model shows acceptable generalization.")

if val_mean[-1] > val_mean[-2]:
    print("📈 Model performance is still improving with more data.")
else:
    print("📊 Model performance has plateaued with current data size.")

## 6. Advanced Analysis <a id="advanced-analysis"></a>

Advanced techniques for deeper insights:
- Student clustering analysis
- Principal Component Analysis
- Prediction intervals
- Model interpretability

In [ ]:
# Student clustering analysis
# **Graph Justification**: Clustering helps identify distinct student personas for targeted interventions

print("=== STUDENT CLUSTERING ANALYSIS ===")

# Prepare data for clustering (use original features, not scaled)
clustering_features = ['comprehension', 'attention_span', 'memory_retention', 'problem_solving', 'critical_thinking',
                      'study_hours_per_week', 'family_support', 'assessment_score']
clustering_data = df[clustering_features]

# Determine optimal number of clusters using elbow method
inertias = []
silhouette_scores = []
k_range = range(2, 11)

from sklearn.metrics import silhouette_score

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(clustering_data)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(clustering_data, kmeans.labels_))

# Plot elbow curve and silhouette scores
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

ax1.plot(k_range, inertias, 'bo-')
ax1.set_title('Elbow Method for Optimal k')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia')
ax1.grid(True, alpha=0.3)

ax2.plot(k_range, silhouette_scores, 'ro-')
ax2.set_title('Silhouette Score vs Number of Clusters')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Choose optimal k (highest silhouette score)
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k} (Silhouette Score: {max(silhouette_scores):.3f})")

# Perform final clustering
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = final_kmeans.fit_predict(clustering_data)

# Add cluster labels to dataframe
df_clustered = df.copy()
df_clustered['cluster'] = cluster_labels

print(f"\nCluster distribution:")
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
for cluster, count in cluster_counts.items():
    print(f"Cluster {cluster}: {count} students ({count/len(df)*100:.1f}%)")

In [ ]:
# Visualize clusters and analyze characteristics
# **Graph Justification**: Cluster visualization helps understand student personas and their characteristics

# PCA for visualization
pca = PCA(n_components=2, random_state=42)
pca_data = pca.fit_transform(clustering_data)

# Create comprehensive cluster analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Student Cluster Analysis', fontsize=16, fontweight='bold')

# 1. PCA visualization of clusters
scatter = axes[0, 0].scatter(pca_data[:, 0], pca_data[:, 1], c=cluster_labels, cmap='viridis', alpha=0.7)
axes[0, 0].set_title('Student Clusters (PCA Visualization)')
axes[0, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[0, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.colorbar(scatter, ax=axes[0, 0])

# 2. Cluster characteristics heatmap
cluster_means = df_clustered.groupby('cluster')[clustering_features].mean()
sns.heatmap(cluster_means.T, annot=True, cmap='RdYlBu_r', center=50, ax=axes[0, 1], fmt='.1f')
axes[0, 1].set_title('Cluster Characteristics Heatmap')
axes[0, 1].set_xlabel('Cluster')

# 3. Assessment score distribution by cluster
df_clustered.boxplot(column='assessment_score', by='cluster', ax=axes[1, 0])
axes[1, 0].set_title('Assessment Score Distribution by Cluster')
axes[1, 0].set_xlabel('Cluster')
axes[1, 0].set_ylabel('Assessment Score')

# 4. Study hours vs performance by cluster
for cluster in range(optimal_k):
    cluster_data = df_clustered[df_clustered['cluster'] == cluster]
    axes[1, 1].scatter(cluster_data['study_hours_per_week'], cluster_data['assessment_score'], 
                      label=f'Cluster {cluster}', alpha=0.7)
axes[1, 1].set_title('Study Hours vs Assessment Score by Cluster')
axes[1, 1].set_xlabel('Study Hours per Week')
axes[1, 1].set_ylabel('Assessment Score')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Analyze cluster characteristics
print("\n=== CLUSTER CHARACTERISTICS ANALYSIS ===")
for cluster in range(optimal_k):
    cluster_data = df_clustered[df_clustered['cluster'] == cluster]
    print(f"\n📊 CLUSTER {cluster} ({len(cluster_data)} students):")
    print(f"   Average Assessment Score: {cluster_data['assessment_score'].mean():.1f}")
    print(f"   Average Study Hours: {cluster_data['study_hours_per_week'].mean():.1f}")
    print(f"   Average Comprehension: {cluster_data['comprehension'].mean():.1f}")
    print(f"   Average Attention Span: {cluster_data['attention_span'].mean():.1f}")
    print(f"   Average Family Support: {cluster_data['family_support'].mean():.1f}")
    
    # Identify cluster persona
    avg_score = cluster_data['assessment_score'].mean()
    avg_study = cluster_data['study_hours_per_week'].mean()
    avg_support = cluster_data['family_support'].mean()
    
    if avg_score >= 75:
        performance = "High Performers"
    elif avg_score >= 50:
        performance = "Average Performers"
    else:
        performance = "Struggling Students"
    
    print(f"   🏷️  Persona: {performance}")

In [ ]:
# Model interpretability and prediction intervals
# **Graph Justification**: Prediction intervals show model uncertainty, crucial for educational decisions

print("=== MODEL INTERPRETABILITY ANALYSIS ===")

# Calculate prediction intervals using bootstrap
def bootstrap_predictions(model, X, n_bootstrap=100):
    """
    Calculate prediction intervals using bootstrap sampling.
    """
    predictions = []
    n_samples = len(X)
    
    for _ in range(n_bootstrap):
        # Bootstrap sample
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_bootstrap = X.iloc[indices]
        
        # Make predictions
        pred = model.predict(X_bootstrap)
        predictions.append(pred)
    
    predictions = np.array(predictions)
    
    # Calculate intervals
    lower = np.percentile(predictions, 2.5, axis=0)
    upper = np.percentile(predictions, 97.5, axis=0)
    mean_pred = np.mean(predictions, axis=0)
    
    return mean_pred, lower, upper

# Calculate prediction intervals for test set
print("Calculating prediction intervals using bootstrap...")
mean_pred, lower_bound, upper_bound = bootstrap_predictions(best_model, X_test, n_bootstrap=50)

# Create prediction interval plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 1. Prediction intervals
sorted_indices = np.argsort(y_test)
ax1.fill_between(range(len(y_test)), lower_bound[sorted_indices], upper_bound[sorted_indices], 
                alpha=0.3, color='blue', label='95% Prediction Interval')
ax1.plot(range(len(y_test)), y_test.iloc[sorted_indices], 'ro', alpha=0.7, label='Actual', markersize=3)
ax1.plot(range(len(y_test)), mean_pred[sorted_indices], 'b-', alpha=0.8, label='Predicted Mean')
ax1.set_title('Prediction Intervals (95% Confidence)')
ax1.set_xlabel('Test Samples (sorted by actual score)')
ax1.set_ylabel('Assessment Score')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Prediction uncertainty vs actual score
uncertainty = upper_bound - lower_bound
ax2.scatter(y_test, uncertainty, alpha=0.6, color='purple')
ax2.set_title('Prediction Uncertainty vs Actual Score')
ax2.set_xlabel('Actual Assessment Score')
ax2.set_ylabel('Prediction Interval Width')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate coverage and interval statistics
coverage = np.mean((y_test >= lower_bound) & (y_test <= upper_bound))
avg_interval_width = np.mean(uncertainty)

print(f"\n=== PREDICTION INTERVAL ANALYSIS ===")
print(f"Coverage (should be ~95%): {coverage:.1%}")
print(f"Average interval width: {avg_interval_width:.2f} points")
print(f"Median interval width: {np.median(uncertainty):.2f} points")

if coverage >= 0.90:
    print("✅ Good coverage - prediction intervals are reliable")
else:
    print("⚠️  Low coverage - prediction intervals may be too narrow")

## 7. Conclusions and Recommendations <a id="conclusions"></a>

Summary of findings and actionable insights for educational stakeholders.

In [ ]:
# Generate comprehensive analysis report
print("=" * 80)
print("🎓 COMPREHENSIVE STUDENT PERFORMANCE ANALYSIS REPORT")
print("=" * 80)

print(f"\n📊 DATASET OVERVIEW:")
print(f"   • Total students analyzed: {len(df):,}")
print(f"   • Features analyzed: {len(selected_features)}")
print(f"   • Average assessment score: {df['assessment_score'].mean():.1f} ± {df['assessment_score'].std():.1f}")
print(f"   • Score range: {df['assessment_score'].min():.1f} - {df['assessment_score'].max():.1f}")

print(f"\n🤖 BEST PERFORMING MODEL:")
print(f"   • Algorithm: {best_model_name}")
print(f"   • Test R² Score: {optimized_r2:.4f} ({optimized_r2*100:.1f}% variance explained)")
print(f"   • Test RMSE: {optimized_rmse:.2f} points")
print(f"   • Cross-validation R²: {best_cv_score:.4f}")

if hasattr(best_model, 'feature_importances_'):
    top_features = feature_importance.head(5)
    print(f"\n🔍 TOP 5 PREDICTIVE FACTORS:")
    for i, (_, row) in enumerate(top_features.iterrows(), 1):
        print(f"   {i}. {row['feature']} (importance: {row['importance']:.3f})")

print(f"\n👥 STUDENT PERSONAS IDENTIFIED:")
for cluster in range(optimal_k):
    cluster_data = df_clustered[df_clustered['cluster'] == cluster]
    avg_score = cluster_data['assessment_score'].mean()
    size = len(cluster_data)
    percentage = size / len(df) * 100
    
    if avg_score >= 75:
        persona = "🌟 High Achievers"
    elif avg_score >= 50:
        persona = "📈 Steady Performers"
    else:
        persona = "🆘 Need Support"
    
    print(f"   • Cluster {cluster}: {persona} - {size} students ({percentage:.1f}%) - Avg Score: {avg_score:.1f}")

print(f"\n📈 KEY INSIGHTS:")

# Calculate key correlations
key_correlations = {
    'Study Hours': df['study_hours_per_week'].corr(df['assessment_score']),
    'Family Support': df['family_support'].corr(df['assessment_score']),
    'Sleep Hours': df['sleep_hours_per_night'].corr(df['assessment_score']),
    'Comprehension': df['comprehension'].corr(df['assessment_score']),
    'Problem Solving': df['problem_solving'].corr(df['assessment_score'])
}

for factor, correlation in sorted(key_correlations.items(), key=lambda x: abs(x[1]), reverse=True):
    direction = "positively" if correlation > 0 else "negatively"
    strength = "strongly" if abs(correlation) > 0.5 else "moderately" if abs(correlation) > 0.3 else "weakly"
    print(f"   • {factor} is {strength} {direction} correlated with performance (r = {correlation:.3f})")

print(f"\n🎯 ACTIONABLE RECOMMENDATIONS:")

# Generate recommendations based on analysis
recommendations = []

if key_correlations['Family Support'] > 0.3:
    recommendations.append("Implement family engagement programs to boost student support systems")

if key_correlations['Sleep Hours'] > 0.2:
    recommendations.append("Educate students about the importance of adequate sleep for academic performance")

if key_correlations['Study Hours'] > 0.2:
    recommendations.append("Provide study skills training and time management workshops")

# Check for at-risk students
at_risk_percentage = len(df[df['assessment_score'] < 40]) / len(df) * 100
if at_risk_percentage > 10:
    recommendations.append(f"Develop targeted intervention programs for {at_risk_percentage:.1f}% of students scoring below 40")

# Technology recommendations
tech_impact = df.groupby('computer_access')['assessment_score'].mean().diff().iloc[-1]
if tech_impact > 5:
    recommendations.append(f"Expand computer access programs (current impact: +{tech_impact:.1f} points)")

for i, rec in enumerate(recommendations, 1):
    print(f"   {i}. {rec}")

print(f"\n⚠️  MODEL LIMITATIONS:")
print(f"   • Prediction intervals average ±{avg_interval_width:.1f} points (95% confidence)")
print(f"   • Model explains {optimized_r2*100:.1f}% of performance variance")
print(f"   • {(1-optimized_r2)*100:.1f}% of variance due to unmeasured factors")
print(f"   • Results based on synthetic data - validate with real student data")

print(f"\n🔄 NEXT STEPS:")
print(f"   1. Validate model with real student performance data")
print(f"   2. Implement continuous monitoring and model updates")
print(f"   3. Develop automated early warning system for at-risk students")
print(f"   4. Create personalized intervention recommendations")
print(f"   5. Establish feedback loop to improve model accuracy")

print("\n" + "=" * 80)
print("📋 ANALYSIS COMPLETE - Ready for Educational Implementation")
print("=" * 80)

In [ ]:
# Save model and results for deployment
import joblib
import json

print("=== SAVING MODEL AND RESULTS ===")

# Save the optimized model
joblib.dump(best_model, 'optimized_student_performance_model.pkl')
print("✅ Saved optimized model to 'optimized_student_performance_model.pkl'")

# Save preprocessing components
joblib.dump(scalers, 'preprocessing_scalers.pkl')
joblib.dump(encoders, 'preprocessing_encoders.pkl')
print("✅ Saved preprocessing components")

# Save feature selection
with open('selected_features.json', 'w') as f:
    json.dump(selected_features, f)
print("✅ Saved selected features list")

# Save analysis results
analysis_results = {
    'model_performance': {
        'best_model': best_model_name,
        'test_r2': float(optimized_r2),
        'test_rmse': float(optimized_rmse),
        'cv_score': float(best_cv_score)
    },
    'feature_importance': feature_importance.to_dict('records') if hasattr(best_model, 'feature_importances_') else None,
    'cluster_analysis': {
        'optimal_clusters': int(optimal_k),
        'cluster_characteristics': cluster_means.to_dict()
    },
    'key_correlations': key_correlations,
    'recommendations': recommendations
}

with open('analysis_results.json', 'w') as f:
    json.dump(analysis_results, f, indent=2)
print("✅ Saved comprehensive analysis results")

print("\n🎉 All components saved successfully!")
print("📁 Files created:")
print("   • optimized_student_performance_model.pkl")
print("   • preprocessing_scalers.pkl")
print("   • preprocessing_encoders.pkl")
print("   • selected_features.json")
print("   • analysis_results.json")

print("\n🚀 Ready for production deployment!")